In [17]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import tweepy
import io
import os
from dotenv import find_dotenv, load_dotenv
import time

In [18]:
df_tae = pd.read_csv('twitter-archive-enhanced.csv')

df_tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1405,699323444782047232,NaN,NaN,2016-02-15 20:04:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""I'm bathing the children what do you want?"" ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/699323444...,10,10,NaN,NaN,NaN,NaN,NaN
2277,667437278097252352,NaN,NaN,2015-11-19 20:20:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,10,10,NaN,NaN,NaN,NaN,NaN
431,820837357901512704,NaN,NaN,2017-01-16 03:37:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Finn. He's wondering if...,8.192277e+17,4.196984e+09,2017-01-11 17:01:16 +0000,https://twitter.com/dog_rates/status/819227688...,12,10,Finn,NaN,NaN,NaN,NaN
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,NaN,NaN,NaN,NaN
1964,673350198937153538,NaN,NaN,2015-12-06 03:56:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bella. She's a Genghis Flopped Canuck....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673350198...,9,10,Bella,NaN,NaN,NaN,NaN


In [19]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

page = requests.get(url)
df_img = pd.read_csv(io.StringIO(page.content.decode('utf-8')), sep='\t')

with open('image-predictions.tsv', 'wb') as file:
    file.write(page.content)

df_img.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1017,709918798883774466,https://pbs.twimg.com/media/CdojYQmW8AApv4h.jpg,2,Pembroke,0.956222,True,Cardigan,0.020727,True,Chihuahua,0.007912,True
2011,879008229531029506,https://pbs.twimg.com/media/DDLdUrqXYAMOVzY.jpg,1,vizsla,0.960513,True,miniature_pinscher,0.009431,True,American_Staffordshire_terrier,0.008711,True
501,675853064436391936,https://pbs.twimg.com/media/CWEcxqWVEAAHyGH.jpg,1,Labrador_retriever,0.868367,True,golden_retriever,0.043305,True,vizsla,0.028207,True
680,683773439333797890,https://pbs.twimg.com/media/CX1AUQ2UAAAC6s-.jpg,1,miniature_pinscher,0.072885,True,Labrador_retriever,0.057866,True,schipperke,0.053257,True
611,680115823365742593,https://pbs.twimg.com/media/CXBBurSWMAELewi.jpg,1,pug,0.999365,True,French_bulldog,0.000544,True,Boston_bull,0.000028,True


In [20]:
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    
# Load credentials from environment variables
bearer_token_ = os.getenv('BEARER_TOKEN')
consumer_key_ = os.getenv('CONSUMER_KEY')
consumer_secret_ = os.getenv('CONSUMER_SECRET')
access_token_ = os.getenv('ACCESS_TOKEN')
access_secret_ = os.getenv('ACCESS_SECRET')

client = tweepy.Client(bearer_token=bearer_token_,
    consumer_key=consumer_key_,
    consumer_secret=consumer_secret_,
    access_token=access_token_,
    access_token_secret=access_secret_,
    wait_on_rate_limit=True
    )

In [ ]:
tweet_ids = df_img['tweet_id'].tolist()

# Process in batches of 100 so it works for the get_tweets method
file = open('tweet_json.txt', 'a')
for i in range(0, len(tweet_ids), 100):
    try:
        tweets = client.get_tweets(tweet_ids[i:i+100], tweet_fields=['public_metrics','text'])
        if tweets.data:
            for tweet in tweets.data:
                file.write(json.dumps(tweet.data) + '\n')
        else:
            print(f"No data returned for {tweet.id}")
            
    except tweepy.TooManyRequests as e:
        print("Rate limit reached. Retrying after 15 mins and 5 secs...")
        time.sleep(15 * 61)  # Wait 15 minutes and 5 secs.

    except Exception as e:
        print("An error occurred: " + str(e))
file.close()

KeyboardInterrupt: 